In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


# -*- coding: utf-8 -*-
"""
Created on Sun Mar  6 15:55:40 2022

@author: 99259
"""
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import package_dataprocess.Nilm_classes
import numpy as np
import keras
# from nilm_ukdale import preprocess
from keras import Model
from keras.models import Sequential
import os
import gc
import pandas as pd
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import PIL.ImageOps
from keras import Model
from keras import layers
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import MaxPooling3D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import GRU, TimeDistributed, Bidirectional
from keras.layers import Embedding
import matplotlib.pyplot as plt

# In[2]:


# 电器id定义
fridge_id = 0
tv_id = 1
kettle_id = 2
microwave_id = 3
washerdryer_id = 4

# 电器开启状态阈值
fridge_threshold = 15
tv_threshold = 15
kettle_threshold = 15
microwave_threshold = 15
washerdryer_threshold = 15

# dp = package_dataprocess.Nilm_classes.Nilm_dataprocess(path_csv="DATA/vicurve_name.xlsx",path_img="DATA/vicurve32x32/")
sum_data_num = np.zeros(100800)

filenames = os.listdir(r'DATA\UKData_by_hour\week2\LowFreq')
# print(filenames)


# In[3]:


boiler_data = np.zeros(100800)
fridge_data = np.zeros(100800)
television_data = np.zeros(100800)
kettle_data = np.zeros(100800)
microwave_data = np.zeros(100800)
washerdryer_data = np.zeros(100800)
dishwasher_data = np.zeros(100800)

# In[4]:


for time in range(7 * 24):
    temp = np.zeros(100800)
    for i in filenames:
        if (i[:10] == str(1452470400 + 3600 * time)):#换周的话记得换一下时间戳喔
            name = i[11:-4]
            name_data = np.load('DATA/UKData_by_hour/week2/LowFreq/' + i)
            name_data = np.array(name_data)
            # print(name_data.shape)
            if len(name_data) > 600:
                name_data = name_data[:600]
            elif len(name_data) < 600:
                name_data = np.pad(name_data, (0, 600 - len(name_data)), 'constant', constant_values=(0, 0))
            name_data = np.where(name_data > 3, name_data, 0)  # 底噪3w

            if (name == 'Fridge freezer0'):
                fridge_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Television0'):
                television_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Kettle0'):
                kettle_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Microwave0'):
                microwave_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Washer dryer0'):
                washerdryer_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Dish washer0'):
                dishwasher_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Boiler0'):
                boiler_data[time * 600:(time + 1) * 600] = name_data

            temp[time * 600:(time + 1) * 600] = name_data
            sum_data_num = sum_data_num + temp
            # print(sum_data_num.shape)
            # sum_data_num = sum_data_num[time*600:(time + 1) * 600] + name_data

# In[5]:


dp = package_dataprocess.Nilm_classes.Nilm_dataprocess(path_csv="DATA/UKData_by_hour/week2/vicurve_name.xlsx",
                                                       path_img="DATA/UKData_by_hour/week2/vicurve32x32/",
                                                       dimension = 7)

# In[6]:


# In[7]:

fridge_data = fridge_data.reshape(fridge_data.shape[0], 1)
television_data = television_data.reshape(television_data.shape[0], 1)
kettle_data = kettle_data.reshape(kettle_data.shape[0], 1)
microwave_data = microwave_data.reshape(microwave_data.shape[0], 1)
washerdryer_data = washerdryer_data.reshape(washerdryer_data.shape[0], 1)
dishwasher_data = dishwasher_data.reshape(dishwasher_data.shape[0], 1)
boiler_data = boiler_data.reshape(boiler_data.shape[0], 1)

# fridge_data = np.load('DATA/UKData_2/Fridge freezer0.npy')
# fridge_data = fridge_data[:100800]
# fridge_data = np.where(fridge_data > dp.air_threshold,fridge_data,0)
#
# television_data = np.load('DATA/UKData_2/Television0.npy')
# television_data = television_data[:100800]
# television_data = np.where(television_data > dp.air_threshold, television_data,0)
#
# kettle_data = np.load('DATA/UKData_2/Kettle0.npy')
# kettle_data = kettle_data[:100800]
# kettle_data = np.where(kettle_data > dp.air_threshold, kettle_data,0)
#
# microwave_data = np.load('DATA/UKData_2/Microwave0.npy')
# microwave_data = microwave_data[:100800]
# microwave_data = np.where(microwave_data > dp.air_threshold, microwave_data,0)
#
# washerdryer_data = np.load('DATA/UKData_2/Washer dryer0.npy')
# washerdryer_data = washerdryer_data[:100800]
# washerdryer_data = np.where(washerdryer_data > dp.air_threshold, washerdryer_data,0)
#
# Fan_data = np.load('DATA/UKData_2/Fan0.npy')
# Fan_data = Fan_data[:100800]
# Fan_data = np.where(fridge_data > dp.air_threshold,Fan_data,0)
#
#

# In[8]:


fridge_labels = dp.create_label(fridge_data, 0, fridge_threshold)
kettle_labels = dp.create_label(kettle_data, 1, kettle_threshold)
microwave_labels = dp.create_label(microwave_data, 2, microwave_threshold)
tv_labels = dp.create_label(television_data, 3, tv_threshold)
washerdryer_labels = dp.create_label(washerdryer_data, 4, washerdryer_threshold)
dishwasher_labels = dp.create_label(dishwasher_data, 5, microwave_threshold)
boiler_labels = dp.create_label(boiler_data, 6, washerdryer_threshold)

sum_label = fridge_labels + tv_labels + kettle_labels + microwave_labels + washerdryer_labels + dishwasher_labels + boiler_labels

mean = sum_data_num[:100800].mean(axis=0)
sum_data = sum_data_num - mean
std = sum_data[:100800].std(axis=0)
sum_data /= std

# In[9]:


sum_data = np.array(sum_data)
print(sum_data.shape)

sum_data = sum_data.reshape(sum_data.shape[0], 1)
print(sum_data.shape)

# In[10]:





# In[11]:


from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import concatenate
from keras.layers import InputLayer

from keras.models import load_model
def my_lossfun(y_true, y_pred):
    y_pred = tf.math.log(y_pred/(1-y_pred))
    #tf.print(y_pred)
    #tf.print(y_true)
    return tf.nn.weighted_cross_entropy_with_logits(labels = y_true, logits = y_pred, pos_weight = tf.constant([1.645, 140.176, 143.294, 5.534, 27.356, 32.949, 1.964]), name=None)
model = load_model("CNNRNNMODEL_CONCA_continue.hdf5", compile=False)
model.compile(loss = my_lossfun,optimizer='rmsprop', metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=[0.5,0.5,0.5,0.5,0.5,0.5,0.5])])

filepath = 'CNNRNNMODEL_CONCA_continue.hdf5'


一切都是命运石之门的选择！
(100800,)
(100800, 1)


In [2]:
confusion = np.zeros((7,4))
temp = np.zeros((7,4))
sum_label_T=sum_label.T
sum_label_T=sum_label_T.astype(int)

In [3]:
test_steps = (128 - 0) // dp.batch_size
for i in range(0,150):
    test_gen = dp.generator_mix(
    data=sum_data,
    label=sum_label,
    min_index=i*128,
    max_index=(i+1)*128
    )
    pred = model.predict_generator(generator = test_gen,steps = test_steps)
    pred = pred.T
    pred[0] = np.where(pred[0] > 0.5,1,0)
    pred[1] = np.where(pred[1] > 0.5,1,0)
    pred[2] = np.where(pred[2] > 0.5,1,0)
    pred[3] = np.where(pred[3] > 0.5,1,0)
    pred[4] = np.where(pred[4] > 0.5,1,0)
    pred[5] = np.where(pred[5] > 0.5,1,0)
    pred[6] = np.where(pred[6] > 0.5,1,0)
    for j in range(7):  
            #print(confusion_matrix(sum_label_T[j][i*128:(i+1)*128],pred[j]).ravel())
            #print(confusion_matrix(sum_label_T[j][i*128:(i+1)*128], pred[j]))
        temp[j][0],temp[j][1],temp[j][2],temp[j][3]=confusion_matrix(sum_label_T[j][i*128:(i+1)*128],pred[j],labels=[1, 0]).ravel()
            #TP FN FP TN
        confusion[j][0]+=temp[j][0]
        confusion[j][1]+=temp[j][1]
        confusion[j][2]+=temp[j][2]
        confusion[j][3]+=temp[j][3]
            

E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.pred

E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.pred

E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.pred

E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.pred

E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
E:\anaconda3\lib\site-packages\keras\engine\training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.pred

In [4]:
print(confusion)

[[2.9680e+03 8.3030e+03 4.8400e+02 1.3845e+04]
 [2.7000e+01 1.9800e+02 2.2370e+03 2.3138e+04]
 [1.7000e+01 2.9400e+02 8.2100e+02 2.4468e+04]
 [7.2300e+02 4.5900e+02 9.0000e+00 2.4409e+04]
 [3.8000e+01 1.0790e+03 4.0200e+02 2.4081e+04]]


In [4]:
print(confusion)#WEEK03
#TP FN FP TN

[[ 4655.  4632.  2938. 13375.]
 [    0.   163.     0. 25437.]
 [    0.   319.     0. 25281.]
 [    0.  1154.     0. 24446.]
 [    0.   534.     0. 25066.]
 [    0.   847.     0. 24753.]
 [    0. 12421.     0. 13179.]]


In [4]:
print(confusion)#WEEK02

[[ 3005.  4671.  1835.  9689.]
 [    0.   142.     0. 19058.]
 [    0.   284.     0. 18916.]
 [    0.  2336.     0. 16864.]
 [    0.     0.     0. 19200.]
 [    0.     0.     0. 19200.]
 [    0.  6540.     0. 12660.]]
